# Actividad 3. Algoritmo _Apriori_
<hr/>

### Nombre: Benjamín Farías V.

**Profesor:** 
- Mauricio Arriagada

**Ayudantes:**
- Felipe Barrientos (fnbarrientos@uc.cl)
- Javier Dreves (jidreves@uc.cl)
- Joaquin Eichholz (jeichholz3@uc.cl)
- Astrid San Martín (aesanmar@uc.cl)


**Antes de comenzar:**

- Laboratorio debe ser realizado **de forma individual**. Obviamente, se pueden discutir ideas, pero cualquier intercambio de códigos **no está permitido**.
- Recuerda orientar tu programación a un paradigma funcional.
- **¡ Comenta todo tu código !**

**Instrucciones de entrega:**

- Debe entregar este laboratorio por `siding`, en sección `cuestionarios`. Descargar archivo ".ipynb" a su equipo y luego subirlo. (También pueden trabajarlo en colab)
- Plazo máximo de entrega: **Martes 3 de Agosto, 10.59 pm.**

**Evaluación:**
- La estructura del código = 0.5
- Salida exitosa = 0.5


<hr/>

**Contexto**

Este laboratorio consiste en estudiar en profundidad e implementar el algoritmo `Apriori`, el cual tiene por
objetivo encontrar `itemsets` frecuentes dentro de una base de datos y generar las reglas de asociación que
superan umbrales de soporte y confianza.
En `data mining`, las reglas de asociación son ampliamente utilizadas para descubrir relaciones entre
variables en bases de datos de gran tamaño [Agrawal et al., 1993]. Aplicaciones clásicas de este tipo de
estrategias pueden ser encontradas en análisis de compras y de características socio-demográficas desde
bases de datos censales.

**Base de Datos**

La base de datos a utilizar corresponde a una parte de la información liberada por `Spotify` para el _RecSys_ Challenge 2018 1 y contiene información de listas de reproducción creadas por usuarios de `Spotify`. Cada lista contiene un número variable de canciones 

**Objetivo**

Para este laboratorio, aplicaremos el algoritmo `a priori` en una base de datos de listas de reproducción publicada por `Spotify`. El objetivo es obtener itemsets de canciones frecuentes y crear reglas de asociación con estos sets. 


<hr/>

## Instrucciones 

1. **Implementar el algoritmo Apriori**: Se espera que el algoritmo sea capaz de retornar a lo menos 3 reglas de asociación entre canciones. El alumno debe ir probando con distintos humbrales  para el la ejecución del código tome un tiempo razonable pero se puedan tener reglas no triviales. 
    - Cargar base de datos
    - Primera pasada: Eliminar todas las listas que contienen canciones que no pasan el umbral
    - Iteraciones posteriores: Generar itemsets con canciones restantes y ver su presencia en las listas que van quedando 


2. **Presentación de reglas**: Mostrar las reglas encontradas y explicar por qué se eligiron. Se deben incluir conceptos de _support_ y _confidence_ de cada regla. 

## Recordar 

- Support = |Itemset| / |T|
    - S = |(pañales, leche, cerveza)|

- Confidence = |Itemset| / |Itemset - regla|
    - C = |(pañales, leche, cerveza)| / |(leche, pañales)|


- **Es de suma importancia que el alumno elabore todo el algoritmo desde cero. No está permitido reutilizar código o usar librerías para la implementación.**

In [120]:
# Se importan las librerías necesarias
import numpy as np

Se carga una parte de la base de datos, para disminuir el tiempo de ejecución:

In [121]:
# Se carga la base de datos, limitando la cantidad de tuplas a analizar para una ejecución más rápida
def load_database(path):
    
    """
    Esta función debe cargar la base de datos de acuerdo a un path. Esta función puede servir también 
    como filtro del número de listas de reproducción a usar para probar a priori.
    """
    db = np.load(path, allow_pickle=True).item()
    database = {x: db[x] for x in db if x <= 500}
    return database

filtered = load_database("spotify.npy")

Se crea el algoritmo A Priori para obtener itemsets frecuentes:

In [122]:
# Esta función se encarga de generar nuevos itemsets de n+1 elementos a partir de los itemsets de n elementos
def generate_itemsets(current_sets, order):
    
    # Se ordenan los itemsets actuales lexicográficamente
    ordered_sets = [sorted(list(x[0]), key=order.get) for x in current_sets]
    
    # Se retornan los que tienen sólo el último elemento distinto y en el correcto orden lexicográfico
    return [set(a + b[-1:]) for a in ordered_sets for b in ordered_sets 
            if (order[a[-1]] < order[b[-1]]) and (a[:-1] == b[:-1])]


# Algoritmo A Priori
def apriori(databa, support):
    
    """
    Función que debe retornar todos los itemsets que cumplen con el support. Puede ser en una estructura de 
    diccionario que entregue el itemset y un contador de apariciones.
    """
    itemsets = {}
    
    # Se cargan todas las canciones a un set inicial 
    songs = set()
    for playlist in databa.values():
        songs |= set(playlist)
    
    # Se genera un diccionario con el orden lexicográfico de las canciones
    order = {}
    index = 0
    for s in songs:
        order[s] = index
        index += 1
    
    # Se crea la lista que contiene a los itemsets de la iteración actual (va cambiando en cada iteración)
    song_sets = [{x} for x in songs]
    
    # Se crea la lista que contiene a las playlists de la iteración actual (se va reduciendo en cada iteración)
    playlists = [set(y) for y in databa.values()]
    
    # Se utiliza un contador para organizar mejor los itemsets frecuentes según su cantidad de items
    counter = 1
    
    # Se itera hasta que se acaben las playlists frecuentes o no se puedan generar más itemsets
    while song_sets and playlists:
        itemsets[counter] = list()
        
        # Cardinalidad de las playlists
        max_number = len(playlists)
        
        # Lista que guarda los itemsets no frecuentes en cada iteración
        non_freq = list()
        
        # Se realiza la comparación entre la frecuencia de los itemsets y el soporte
        for s in song_sets:
            amount = sum(1 for v in playlists if s.issubset(v))
            support_number = amount / max_number
            if support_number >= support:
                itemsets[counter].append((s, amount))
            else:
                non_freq.append(s)
        
        # Se eliminan las playlists que contienen a algún itemset no frecuente
        new_playlists = list()
        for z in playlists:
            add = True
            for x in non_freq:
                if x.issubset(z):
                    add = False
            if add:
                new_playlists.append(z)
        playlists = new_playlists
        
        # Se generan los nuevos itemsets de tamaño n+1 para la siguiente iteración
        song_sets = generate_itemsets(itemsets[counter], order)
        counter += 1
    return itemsets

results = apriori(filtered, 0.01)

A continuación se buscan reglas de confianza entre los itemsets de tamaño n y n-1:

In [123]:
# Para obtener reglas se utilizó un criterio basado en la confianza, donde se busca que esta sea lo mayor posible
def rules(itemsets, confidence):
    
    """
    Retorna las reglas que cumplen con la confianza pedida.
    """
    # Lista de reglas
    rules_list = list()
    
    # Se itera en un loop, partiendo desde los itemsets de mayor tamaño
    index = len(itemsets)
    while index > 1:
        
        # Corresponde a los itemsets de tamaño n
        for it1 in itemsets[index]:
            
            # Corresponde a los itemsets de tamaño n-1
            for it2 in itemsets[index - 1]:
                
                # Se revisa si el itemset menor es subconjunto del mayor, 
                # ignorando los casos en los que no se cumpla (no estarían relacionados)
                if it2[0].issubset(it1[0]):
                    
                    # Se calcula la confianza
                    current_confidence = it1[1] / it2[1]
                    
                    # Si se cumple el criterio se agrega la asociación a la lista de reglas
                    if current_confidence >= confidence:
                        original = it2[0]
                        implied = it1[0] - it2[0]
                        rules_list.append(f'{original} -> {implied}')
        index -= 1
    return rules_list

rules(results, 0.9)

["{'Heathens', 'Sucker For Pain (with Wiz Khalifa, Imagine Dragons, Logic & Ty Dolla $ign feat. X Ambassadors)', 'Lane Boy', 'The Judge', 'Ride', 'Car Radio'} -> {'All Time Low'}",
 "{'Heathens', 'Sucker For Pain (with Wiz Khalifa, Imagine Dragons, Logic & Ty Dolla $ign feat. X Ambassadors)', 'Lane Boy', 'All Time Low', 'The Judge', 'Ride'} -> {'Car Radio'}",
 "{'Heathens', 'Sucker For Pain (with Wiz Khalifa, Imagine Dragons, Logic & Ty Dolla $ign feat. X Ambassadors)', 'Lane Boy', 'All Time Low', 'Ride', 'Car Radio'} -> {'The Judge'}",
 "{'Heathens', 'Sucker For Pain (with Wiz Khalifa, Imagine Dragons, Logic & Ty Dolla $ign feat. X Ambassadors)', 'All Time Low', 'The Judge', 'Ride', 'Car Radio'} -> {'Lane Boy'}",
 "{'Heathens', 'Lane Boy', 'All Time Low', 'The Judge', 'Ride', 'Car Radio'} -> {'Sucker For Pain (with Wiz Khalifa, Imagine Dragons, Logic & Ty Dolla $ign feat. X Ambassadors)'}",
 "{'Heathens', 'Sucker For Pain (with Wiz Khalifa, Imagine Dragons, Logic & Ty Dolla $ign feat.

<hr/>
<center> <h1>Fin del laboratorio.</h1> </center>
​